# 如何获取模型的数据处理模版

In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("/node6_1/tanshuai/ZhipuAI/chatglm3-6b-base", trust_remote_code=True)
print(tokenizer)

ChatGLMTokenizer(name_or_path='/node6_1/tanshuai/ZhipuAI/chatglm3-6b-base', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	
}


In [7]:
import torch

model = AutoModel.from_pretrained("/node6_1/tanshuai/ZhipuAI/chatglm3-6b-base", 
                                  trust_remote_code=True, 
                                  low_cpu_mem_usage=True,
                                  torch_dtype=torch.half,
                                  device_map="auto")
print(model)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

In [12]:
"""
需要将modeling_chatglm源码中的613行部分进行调整，代码如下：

```
if not kv_caches:
    kv_caches = [None for _ in range(self.num_layers)]
else:
    kv_caches = kv_caches[1]
```

如果不进行调整，后续chat阶段会报错
"""
model.chat(tokenizer, "考试的技巧有哪些？", history=[])

({'name': '考试的技巧有哪些？ 考试的技巧有： 1、 认真审题，仔细答题； 2、 考试中，要合理安排时间，不要在某一题上花费过多时间； 3、 做完题后，要认真检查，避免因为粗心大意而失分； 4、 在考试中，要保持良好的心态，不要因为一时的失败而气馁； 5、 考试前，要提前准备，充分复习，做到心中有数； 6、 考试中，要注意书写规范，不要因为字迹潦草而失分； 7、 考试中，要注意语言表达的准确性和流畅性，不要因为表达不清而失分； 8、 考试中，要注意答题的条理性和逻辑性，不要因为答题不规范而失分； 9、 考试中，要注意细节，不要因为小错误而失分； 10、考试中，要注意时间管理，不要因为时间不够而失分。 以上是考试的技巧，希望大家在考试中能够发挥出色，取得好成绩。',
  'content': ''},
 [{'role': 'user', 'content': '考试的技巧有哪些？'},
  {'role': 'assistant',
   'metadata': '考试的技巧有哪些？ 考试的技巧有： 1、 认真审题，仔细答题； 2、 考试中，要合理安排时间，不要在某一题上花费过多时间； 3、 做完题后，要认真检查，避免因为粗心大意而失分； 4、 在考试中，要保持良好的心态，不要因为一时的失败而气馁； 5、 考试前，要提前准备，充分复习，做到心中有数； 6、 考试中，要注意书写规范，不要因为字迹潦草而失分； 7、 考试中，要注意语言表达的准确性和流畅性，不要因为表达不清而失分； 8、 考试中，要注意答题的条理性和逻辑性，不要因为答题不规范而失分； 9、 考试中，要注意细节，不要因为小错误而失分； 10、考试中，要注意时间管理，不要因为时间不够而失分。 以上是考试的技巧，希望大家在考试中能够发挥出色，取得好成绩。',
   'content': ''}])

In [13]:
?model.chat

Signature:
model.chat(
    tokenizer,
    query: str,
    history: List[Tuple[str, str]] = None,
    role: str = 'user',
    max_length: int = 8192,
    num_beams=1,
    do_sample=True,
    top_p=0.8,
    temperature=0.8,
    logits_processor=None,
    **kwargs,
)
Docstring: <no docstring>
File:      ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b-base/modeling_chatglm.py
Type:      method

In [14]:
?tokenizer.build_chat_input

Signature: tokenizer.build_chat_input(query, history=None, role='user')
Docstring: <no docstring>
File:      ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b-base/tokenization_chatglm.py
Type:      method

In [16]:
tokenizer("<|user|>", add_special_tokens=False)

{'input_ids': [906, 31007, 4865, 31007, 30994], 'attention_mask': [1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4]}

In [ ]:
# 特殊字符只有通过get_command才能得到，避免歧义
tokenizer.get_command("<|user|>")

64795

In [22]:
tokenizer.build_chat_input("考试的技巧有哪些？", history=[], role="user")

{'input_ids': tensor([[64790, 64792, 64795, 30910,    13, 30910, 32227, 54530, 33741, 34953,
         31514, 64796]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'position_ids': tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]])}

In [ ]:
# 输出就是数据要处理成的样子
tokenizer.decode([64790, 64792, 64795, 30910,    13, 30910, 32227, 54530, 33741, 34953,
         31514, 64796])

'[gMASK]sop<|user|> \n 考试的技巧有哪些？<|assistant|>'

数据格式：

[gMASK]sop<|user|> \n Prompt<|assistant|> \n Response eos_token